<a href="https://colab.research.google.com/github/aicreativeexplorer/YT-Automation/blob/main/YT_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# === AUTO-RUN HEARTBEAT (every 60 minutes) ===
import threading, time, IPython

HEARTBEAT_INTERVAL = 60 * 60   # 60 minutes

def heartbeat_loop():
    while True:
        try:
            print("\n❤️  Heartbeat triggered — auto-running keepalive cell...")
            IPython.display.display(IPython.display.Javascript(
                'google.colab.kernel.invokeFunction("keepalive", [], {});'
            ))
        except Exception as e:
            print("⚠️ Heartbeat error:", e)
        time.sleep(HEARTBEAT_INTERVAL)

def start_heartbeat():
    t = threading.Thread(target=heartbeat_loop, daemon=True)
    t.start()
    print("🔥 Auto-run heartbeat started (interval = 60 min).")

# Register a hidden keepalive callback
from google.colab import output
def _keepalive():
    print("⏳ Notebook auto-ran keepalive at", time.ctime())
output.register_callback("keepalive", _keepalive)

start_heartbeat()



❤️  Heartbeat triggered — auto-running keepalive cell...🔥 Auto-run heartbeat started (interval = 60 min).



<IPython.core.display.Javascript object>

⏳ Notebook auto-ran keepalive at Thu Nov 20 20:05:00 2025


In [46]:
start_autosave(interval_sec=300)  # every 5 minutes snapshot

Started autosave thread: interval 300 sec


In [45]:
# AUTO-PERSIST STARTUP CELL — paste & run at top of notebook (one cell)
# - mounts Drive
# - auto-restores uploaded sample from /mnt/data (if present) into /content and Drive
# - provides save(file) and start_autosave(interval_sec) helpers
# - optional: simple GitHub push helper for small files (needs GITHUB_TOKEN env)

import os, shutil, time, threading
from pathlib import Path
from google.colab import drive

# ---------- CONFIG ----------
# change these if you want a different Drive folder
DRIVE_BASE = Path("/content/drive/MyDrive/AI-Automation")
DRIVE_SAMPLE = DRIVE_BASE / "sample.mp4"
DRIVE_CHECKPOINTS = DRIVE_BASE / "checkpoints"
DRIVE_OUTPUTS = DRIVE_BASE / "outputs"
LOCAL_CONTENT_DIR = Path("/content")
LOCAL_SAMPLE_TARGET = LOCAL_CONTENT_DIR / "sample_from_upload.mp4"

# known uploaded path from this session (auto-detect fallback)
POSSIBLE_UPLOAD_PATHS = [
    Path("/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"),
    Path("/content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"),
]

# ---------- Mount Drive ----------
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)
DRIVE_BASE.mkdir(parents=True, exist_ok=True)
DRIVE_OUTPUTS.mkdir(parents=True, exist_ok=True)
DRIVE_CHECKPOINTS.mkdir(parents=True, exist_ok=True)
print("Drive mounted at /content/drive")

# ---------- Auto-detect uploaded mp4 and copy to Drive + /content ----------
def find_uploaded():
    # check known locations first
    for p in POSSIBLE_UPLOAD_PATHS:
        if p.exists():
            return p
    # fallback: search /content and /mnt for any mp4
    for root in [Path("/content"), Path("/mnt")]:
        if root.exists():
            mp4s = list(root.rglob("*.mp4"))
            if mp4s:
                return mp4s[0]
    return None

uploaded = find_uploaded()
if uploaded:
    print("Found uploaded file:", uploaded)
    # copy into /content (so pipeline expects it there)
    try:
        shutil.copy2(str(uploaded), str(LOCAL_SAMPLE_TARGET))
        print("Copied uploaded file ->", LOCAL_SAMPLE_TARGET)
    except Exception as e:
        print("Warn: failed to copy to /content:", e)
    # copy into Drive sample path if not already present or newer
    try:
        if (not DRIVE_SAMPLE.exists()) or (uploaded.stat().st_mtime > DRIVE_SAMPLE.stat().st_mtime):
            shutil.copy2(str(uploaded), str(DRIVE_SAMPLE))
            print("Copied uploaded file -> Drive:", DRIVE_SAMPLE)
        else:
            print("Drive sample already exists and is newer. Skipping Drive copy.")
    except Exception as e:
        print("Warn: failed to copy to Drive:", e)
else:
    print("No uploaded mp4 found in standard locations. If you have a sample in Drive, it will be used when needed.")

# ---------- Helper: save(file) -> copies file to Drive outputs with timestamp ----------
def save_to_drive(local_path, dest_name=None):
    local_path = Path(local_path)
    if not local_path.exists():
        raise FileNotFoundError(f"Local file not found: {local_path}")
    ts = int(time.time())
    dest = DRIVE_OUTPUTS / (dest_name or f"{local_path.stem}_{ts}{local_path.suffix}")
    shutil.copy2(str(local_path), str(dest))
    print("Saved to Drive:", dest)
    return dest

# ---------- Optional: GitHub push helper for small files (100MB limit) ----------
def git_push_small(file_path, repo="aicreativeexplorer/YT-Automation", branch="main"):
    """
    Push a small file to your GitHub repo. Requires env var GITHUB_TOKEN set in session.
    Use: import os; os.environ['GITHUB_TOKEN']="ghp_xxx" in a private cell before calling.
    """
    token = os.environ.get("GITHUB_TOKEN")
    if not token:
        raise EnvironmentError("Set GITHUB_TOKEN env var (repo push).")
    tmp = Path("/tmp/git_push_repo")
    if tmp.exists():
        shutil.rmtree(tmp)
    tmp.mkdir()
    # clone and copy, commit
    os.system(f"git clone https://{token}@github.com/{repo}.git {tmp}")
    shutil.copy2(str(file_path), str(tmp / Path(file_path).name))
    cur = os.getcwd()
    os.chdir(tmp)
    os.system('git config user.email "colab@local"')
    os.system('git config user.name "ColabAuto"')
    os.system(f'git add {Path(file_path).name} && git commit -m "Add {Path(file_path).name}" || echo "nothing to commit"')
    os.system(f'git push origin {branch} || echo "push failed"')
    os.chdir(cur)
    print("Attempted push to GitHub (check repo).")

# ---------- Autosave thread: sync /content -> Drive every N seconds ----------
_autosave_thread = None
_autosave_stop = threading.Event()

def _autosave_loop(interval_sec=60, paths_to_sync=None):
    paths_to_sync = paths_to_sync or ["/content"]
    while not _autosave_stop.is_set():
        try:
            ts = int(time.time())
            backup_dir = DRIVE_BASE / "session_backups" / f"backup_{ts}"
            backup_dir.mkdir(parents=True, exist_ok=True)
            for p in paths_to_sync:
                src = Path(p)
                if not src.exists():
                    continue
                # copy only top-level files (fast) and small dirs; avoid huge copy every loop
                for f in src.glob("*"):
                    # skip Drive mount itself to avoid recursion
                    if "/content/drive" in str(f):
                        continue
                    # only copy files under 200MB by default (safe)
                    try:
                        if f.is_file() and f.stat().st_size < 200 * 1024 * 1024:
                            shutil.copy2(str(f), str(backup_dir / f.name))
                        # small dir handling: copy single-level files
                        elif f.is_dir():
                            for sf in f.glob("*"):
                                if sf.is_file() and sf.stat().st_size < 200 * 1024 * 1024:
                                    destd = backup_dir / f.name
                                    destd.mkdir(parents=True, exist_ok=True)
                                    shutil.copy2(str(sf), str(destd / sf.name))
                    except Exception as e:
                        print("Autosave copy warning for", f, e)
            print(f"[autosave] synced snapshot to {backup_dir}")
        except Exception as e:
            print("Autosave loop error:", e)
        _autosave_stop.wait(interval_sec)

def start_autosave(interval_sec=120, paths_to_sync=None):
    global _autosave_thread, _autosave_stop
    if _autosave_thread and _autosave_thread.is_alive():
        print("Autosave already running.")
        return
    _autosave_stop.clear()
    _autosave_thread = threading.Thread(target=_autosave_loop, args=(interval_sec, paths_to_sync), daemon=True)
    _autosave_thread.start()
    print("Started autosave thread: interval", interval_sec, "sec")

def stop_autosave():
    global _autosave_thread, _autosave_stop
    if _autosave_thread and _autosave_thread.is_alive():
        _autosave_stop.set()
        _autosave_thread.join(timeout=5)
        print("Stopped autosave thread.")
    else:
        print("No autosave running.")

# ---------- Quick usage hints printed ----------
print("\nREADY — auto-persistence enabled.")
print("Detected sample (if any):", uploaded or "NONE")
print("Local sample target:", LOCAL_SAMPLE_TARGET)
print("Drive sample path  :", DRIVE_SAMPLE)
print("Drive outputs path :", DRIVE_OUTPUTS)
print("\nFunctions you can call:")
print("  save_to_drive(local_path)   # copy file to Drive outputs")
print("  start_autosave(interval_sec=120)  # begin background autosave of /content -> Drive")
print("  stop_autosave()             # stop background autosave")
print("  git_push_small(local_file)  # optional small-file push (requires GITHUB_TOKEN env)")


Mounting Google Drive...
Mounted at /content/drive
Drive mounted at /content/drive
Found uploaded file: /content/drive_backup/MyDrive/AI-Automation/outputs/final_short_1763667365.mp4
Copied uploaded file -> /content/sample_from_upload.mp4
Copied uploaded file -> Drive: /content/drive/MyDrive/AI-Automation/sample.mp4

READY — auto-persistence enabled.
Detected sample (if any): /content/drive_backup/MyDrive/AI-Automation/outputs/final_short_1763667365.mp4
Local sample target: /content/sample_from_upload.mp4
Drive sample path  : /content/drive/MyDrive/AI-Automation/sample.mp4
Drive outputs path : /content/drive/MyDrive/AI-Automation/outputs

Functions you can call:
  save_to_drive(local_path)   # copy file to Drive outputs
  start_autosave(interval_sec=120)  # begin background autosave of /content -> Drive
  stop_autosave()             # stop background autosave
  git_push_small(local_file)  # optional small-file push (requires GITHUB_TOKEN env)


In [47]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [43]:
%%bash
# 1) Inspect what's currently in /content/drive
echo "Contents of /content/drive BEFORE fix:"
ls -la /content/drive || true
echo "----"

# 2) If non-empty, move it to a safe backup folder instead of deleting
mkdir -p /content/drive_backup || true
if [ "$(ls -A /content/drive 2>/dev/null)" ]; then
  echo "Moving existing /content/drive/* -> /content/drive_backup/"
  mv /content/drive/* /content/drive_backup/ 2>/dev/null || true
  echo "Moved. Backup dir: /content/drive_backup/"
else
  echo "/content/drive is already empty."
fi
echo "----"

# 3) Ensure mountpoint dir exists and is empty
rm -rf /content/drive
mkdir -p /content/drive


Contents of /content/drive BEFORE fix:
total 12
drwxr-xr-x 3 root root 4096 Nov 20 18:37 .
drwxr-xr-x 1 root root 4096 Nov 20 18:37 ..
drwxr-xr-x 3 root root 4096 Nov 20 18:37 MyDrive
----
Moving existing /content/drive/* -> /content/drive_backup/
Moved. Backup dir: /content/drive_backup/
----


In [48]:
%%bash
SRC="/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

if [ -f "$SRC" ]; then
  mkdir -p /content/drive/MyDrive/AI-Automation
  cp "$SRC" /content/drive/MyDrive/AI-Automation/sample.mp4
  cp "$SRC" "/content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"
  echo "Copied uploaded file to Drive and /content."
else
  echo "WARNING: Uploaded file missing at $SRC"
fi


In [44]:
# CONFIG — edit only if you moved paths
REPO_DIR = "/content/YT-Automation"
NOTEBOOK_NAME = "YT-Automation.ipynb"
UPLOADED_VIDEO = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"  # your uploaded demo
DRIVE_TOKEN_PATH = "/content/drive/MyDrive/AI-Automation/hf_token.txt"  # put HF token here
OUTPUT_DRIVE_FOLDER = "/content/drive/MyDrive/AI-Automation/outputs/stitched"
CHECKPOINT_DIR = "/content/drive/MyDrive/AI-Automation/checkpoints"
SVD_VERSION = "svd"  # 'svd' (open) or 'svd-xt-1-1' (better but gated)
USE_AUTO_PUSH = False  # no auto-push by default
print("CONFIG OK")


CONFIG OK


In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Auto-login to HuggingFace if token present in Drive
import os
if os.path.exists(DRIVE_TOKEN_PATH):
    from huggingface_hub import login
    with open(DRIVE_TOKEN_PATH,'r') as f:
        token = f.read().strip()
    login(token=token)
    print("Logged into HuggingFace from Drive token.")
else:
    print("No HF token at", DRIVE_TOKEN_PATH, "- you'll be asked if downloading gated models.")


Mounted at /content/drive
Logged into HuggingFace from Drive token.


In [7]:
%%bash
# minimal set used by pipeline
pip install -q kornia open_clip_torch timm transformers safetensors accelerate pytorch-lightning einops imageio-ffmpeg imwatermark opencv-python-headless==4.6.0.66 gradio==3.34.0
# RIFE + Real-ESRGAN notebooks will install their deps inside their cells when needed.
echo "installed deps"


installed deps


In [8]:
%%bash
# clone once per session
if [ ! -d /content/generative-models ]; then
  git clone https://github.com/Stability-AI/generative-models.git /content/generative-models
fi
# small helper scripts (won't break if run multiple times)
mkdir -p /content/scripts
cat > /content/scripts/ffmpeg_stitch.sh <<'SH'
#!/usr/bin/env bash
# Usage: ./ffmpeg_stitch.sh out.mp4 file1.mp4 file2.mp4 ...
out="$1"; shift
# convert to ts and concat
tsfiles=""
for f in "$@"; do
  ts="/tmp/$(basename "$f").ts"
  ffmpeg -y -i "$f" -c copy -bsf:v h264_mp4toannexb -f mpegts "$ts"
  tsfiles="${tsfiles}|${ts}"
done
tsfiles=${tsfiles#|}
ffmpeg -y -i "concat:${tsfiles}" -c copy -bsf:a aac_adtstoasc "$out"
SH
chmod +x /content/scripts/ffmpeg_stitch.sh
echo "cloned generative-models; helper scripts in /content/scripts"


cloned generative-models; helper scripts in /content/scripts


Cloning into '/content/generative-models'...


In [9]:
import os, subprocess, uuid, json, shutil, time
from pathlib import Path

def run_cmd(cmd, quiet=False):
    print("RUN:", cmd)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if not quiet:
        print(p.stdout)
        if p.stderr:
            print("ERR:", p.stderr[:1000])
    return p

def ensure_folder(p):
    Path(p).mkdir(parents=True, exist_ok=True)
    return p

ensure_folder(OUTPUT_DRIVE_FOLDER)
ensure_folder(CHECKPOINT_DIR)

# ---- SVD: wrapper (placeholder) ----
def generate_svd_from_text(prompt, seed=None, num_frames=14, out_prefix="svd"):
    """
    Minimal wrapper to call SVD sampling. The heavy work is in the SVD notebook code.
    Here we assume `sample()` exists in session OR we call a local script.
    For now this function writes a JSON config file for the manual SVD notebook to pick up.
    """
    jobid = f"{out_prefix}_{int(time.time())}_{uuid.uuid4().hex[:6]}"
    jobdir = f"/tmp/{jobid}"
    ensure_folder(jobdir)
    cfg = {
        "prompt": prompt,
        "seed": seed or "random",
        "num_frames": num_frames,
        "out": f"{jobdir}/{jobid}.mp4"
    }
    cfg_path = f"{jobdir}/cfg.json"
    with open(cfg_path,'w') as f: json.dump(cfg,f)
    print("SVD job written:", cfg_path)
    # NOTE: actual execution will be in the notebook cell that imports the SVD sampling code
    return cfg["out"], jobdir

# ---- RIFE: interpolation wrapper ----
def interpolate_with_rife(infile, factor=4):
    out = f"{Path(infile).with_suffix('')}_rife.mp4"
    # Call public RIFE Colab? For now we use a simple ffmpeg fps trick as placeholder
    run_cmd(f"ffmpeg -y -i '{infile}' -filter:v 'minterpolate=fps=30:mi_mode=mci:mc_mode=aobmc:me_mode=bidir:vsbmc=1' -c:v libx264 -crf 18 '{out}'")
    return out

# ---- Real-ESRGAN: upscale wrapper (placeholder) ----
def upscale_realesrgan(infile, scale=2):
    out = f"{Path(infile).with_suffix('')}_up.mp4"
    # placeholder: re-encode at higher resolution (actual Real-ESRGAN model would process frames)
    run_cmd(f"ffmpeg -y -i '{infile}' -vf scale=iw*{scale}:ih*{scale} -c:v libx264 -crf 18 '{out}'")
    return out

# ---- Stitch helper ----
def stitch_files(inputs, out):
    inputs = [str(i) for i in inputs]
    run_cmd(f"/content/scripts/ffmpeg_stitch.sh '{out}' " + " ".join([f"'{i}'" for i in inputs]))
    return out

# ---- TTS helper (ElevenLabs fallback) ----
def tts_generate(text, outfile="/tmp/tts.wav", provider="free"):
    """
    provider: 'elevenlabs' (needs key), 'gtts' (free).
    This is a simple wrapper: I recommend hooking to ElevenLabs for quality or use gTTS for free.
    """
    if provider=="gtts":
        # install gTTS if needed
        run_cmd("python -m pip install -q gTTS pydub")
        from gtts import gTTS
        tts = gTTS(text=text, lang='en')
        tts.save(outfile)
        return outfile
    else:
        # try gTTS as default
        run_cmd("python -m pip install -q gTTS pydub")
        from gtts import gTTS
        tts = gTTS(text=text, lang='en')
        tts.save(outfile)
        return outfile

# ---- Caption burn-in (ffmpeg) ----
def burn_captions(video_in, text, out):
    # naive single-line centered caption
    cmd = (
        f'ffmpeg -y -i "{video_in}" -vf '
        f"\"drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf: text='{text}':"
        "fontcolor=white:fontsize=48:box=1:boxcolor=0x00000099:boxborderw=5:x=(w-text_w)/2:y=h-150\" "
        f'-c:a copy "{out}"'
    )
    run_cmd(cmd)
    return out

print("Helpers loaded.")


Helpers loaded.


In [10]:
# TEXT-ONLY MODE — create config and call SVD job (sample wrapper)
prompt = "9:16 vertical, ultra-detailed close-up of a tiny mechanical fox exploring a sunlit garden, cinematic shallow depth-of-field, warm color grade, soft motion. No text or logos."
out_mp4, jobdir = generate_svd_from_text(prompt, seed=1234, num_frames=25, out_prefix="tinyfox")
print("SVD job created. Local jobdir:", jobdir)
print("Once SVD sampling cell runs, output will be:", out_mp4)


SVD job written: /tmp/tinyfox_1763639622_3c71ba/cfg.json
SVD job created. Local jobdir: /tmp/tinyfox_1763639622_3c71ba
Once SVD sampling cell runs, output will be: /tmp/tinyfox_1763639622_3c71ba/tinyfox_1763639622_3c71ba.mp4


In [3]:
from google.colab import files
uploaded = files.upload()


Saving Kling AI- Next-Gen AI Video & AI Image Generator.mp4 to Kling AI- Next-Gen AI Video & AI Image Generator.mp4


In [4]:
import os
os.listdir()

['.config',
 'Kling AI- Next-Gen AI Video & AI Image Generator.mp4',
 'sample_data']

In [11]:
# IMAGE-TO-VIDEO MODE
# Use your uploaded file to extract a conditioning frame (change timestamp if you want)
import os
frame_path = "/content/frame_for_svd.png"
run_cmd(f"ffmpeg -y -ss 00:00:01 -i '{UPLOADED_VIDEO}' -frames:v 1 '{frame_path}'")
print("Extracted frame:", frame_path)

# Now create SVD job that uses this image
prompt_image = "9:16 vertical, mechanical fox exploring, use this frame as a style reference. No text."
out_mp4_img, jobdir_img = generate_svd_from_text(prompt_image, seed=999, num_frames=25, out_prefix="img2vid")
# attach the frame info to jobdir so the sampling cell can pick it
open(f"{jobdir_img}/cond_frame.txt","w").write(frame_path)
print("Image->video job created at", jobdir_img, "expected out:", out_mp4_img)


RUN: ffmpeg -y -ss 00:00:01 -i '/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4' -frames:v 1 '/content/frame_for_svd.png'

ERR: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enabl

In [20]:
!apt-get install -y fonts-dejavu-core


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-dejavu-core
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,041 kB of archives.
After this operation, 3,025 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Fetched 1,041 kB in 0s (4,691 kB/s)
Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../fonts-dejavu-core_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-core (2.37-2build1) ...
Setting up fonts-dejavu-core (2.37-2build1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [5]:
!apt-get update -qq && apt-get install -y -qq fonts-dejavu-core


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [1]:
!ffmpeg -version
!ffmpeg -hide_banner -filters | grep -E 'drawtext|minterpolate' || true


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [2]:
%%bash
JOBDIR="/tmp/svd_job_fix_$(date +%s)"
mkdir -p "$JOBDIR"
OUT="$JOBDIR/simulated_svd.mp4"
IN="/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

ffmpeg -y -i "$IN" -t 6 \
  -vf "scale=720:1280:force_original_aspect_ratio=decrease,pad=720:1280:(ow-iw)/2:(oh-ih)/2,drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf:text='SIMULATED SVD':fontsize=28:fontcolor=white:x=20:y=40" \
  -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart "$OUT"

echo "DONE -> $OUT"
ls -lh "$OUT" || true


DONE -> /tmp/svd_job_fix_1763661677/simulated_svd.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [4]:
import os, subprocess, uuid, time
from pathlib import Path

# Your uploaded file
IN = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

# Create job folder
job_id = f"svd_job_{int(time.time())}_{uuid.uuid4().hex[:6]}"
jobdir = Path(f"/tmp/{job_id}")
jobdir.mkdir(parents=True, exist_ok=True)

OUT = jobdir / f"{job_id}.mp4"

print("Creating simulated SVD output WITHOUT lavfi source...")
print("Input:", IN)
print("Output:", OUT)

# 1) extract a single frame as PNG
frame = jobdir / "frame.png"
cmd1 = f"ffmpeg -y -i '{IN}' -vframes 1 '{frame}'"
subprocess.run(cmd1, shell=True, check=False)

# 2) make a 2-second video by looping the frame
cmd2 = (
    f"ffmpeg -y -loop 1 -i '{frame}' -t 2 "
    f"-c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast "
    f"-movflags +faststart '{OUT}'"
)
subprocess.run(cmd2, shell=True, check=False)

print("\nDONE ✔")
print("Simulated SVD output created at:")
print(str(OUT))


Creating simulated SVD output WITHOUT lavfi source...
Input: /content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4
Output: /tmp/svd_job_1763661781_b1fe6c/svd_job_1763661781_b1fe6c.mp4

DONE ✔
Simulated SVD output created at:
/tmp/svd_job_1763661781_b1fe6c/svd_job_1763661781_b1fe6c.mp4


In [5]:
# SVD sampling runner (paste into a cell and run after you confirm model & weights)
# ONE-CELL: Robust fix for "No SVD job config found in /tmp"
# - creates a /tmp/svd_* job with cfg.json if none exists
# - tries to call sample() if available; otherwise produces simulated mp4
# - safe, idempotent, prints everything you need
import os, json, time, uuid, subprocess
from pathlib import Path
from shlex import quote

# ---------- USER / ENV SETTINGS ----------
# Use your uploaded file path (developer-provided)
UPLOADED_FILE = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

# default prompt (edit inline if you want different)
DEFAULT_PROMPT = (
    "9:16 vertical, ultra-detailed close-up of a tiny mechanical fox exploring a sunlit garden, "
    "cinematic shallow depth-of-field, warm color grade, subtle camera push-in, photorealistic textures, no text or logos."
)

DEFAULT_SEED = 1234
DEFAULT_NUM_FRAMES = 25

# Where SVD checkpoints are expected (adjust if you use Drive)
if "CHECKPOINT_DIR" not in globals():
    CHECKPOINT_DIR = "/content/checkpoints"

if "SVD_VERSION" not in globals():
    # keep as 'svd' if you only have open weights; use 'svd-xt-1-1' only if you have access
    SVD_VERSION = "svd-xt-1-1"

# ---------- helpers ----------
def run_cmd(cmd, fail_ok=False):
    print("RUN:", cmd)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.stdout:
        print(p.stdout.strip())
    if p.stderr:
        # only show trimmed error to avoid spamming
        print("ERR:", p.stderr.strip()[:1000])
    if p.returncode != 0 and not fail_ok:
        raise RuntimeError(f"Command failed: {cmd}\nstderr: {p.stderr}")
    return p

def find_latest_jobcfg(tmpdir="/tmp"):
    p = Path(tmpdir)
    candidates = sorted(p.glob("svd_*"), key=lambda x: x.stat().st_mtime, reverse=True)
    for d in candidates:
        cfg = d / "cfg.json"
        if cfg.exists():
            return str(cfg)
    return None

# ---------- main logic ----------
print("Looking for existing SVD job cfg in /tmp ...")
job_cfg = find_latest_jobcfg("/tmp")

if job_cfg:
    print("Found job config:", job_cfg)
else:
    # create a new job folder and cfg
    jobid = f"svd_job_{int(time.time())}_{uuid.uuid4().hex[:6]}"
    jobdir = Path("/tmp") / jobid
    jobdir.mkdir(parents=True, exist_ok=True)
    out_mp4 = str(jobdir / f"{jobid}.mp4")
    cfg = {
        "prompt": DEFAULT_PROMPT,
        "seed": DEFAULT_SEED,
        "num_frames": DEFAULT_NUM_FRAMES,
        "out": out_mp4,
        "conditioning_image": UPLOADED_FILE  # if you want image->video
    }
    cfg_path = jobdir / "cfg.json"
    with open(cfg_path, "w", encoding="utf8") as f:
        json.dump(cfg, f, indent=2)
    job_cfg = str(cfg_path)
    print("WROTE new SVD job config to:", job_cfg)
    print("EXPECTED OUTPUT ->", out_mp4)
    # create a short simulated MP4 so downstream pipeline can run
    print("Creating simulated sample MP4 (2s) — so RIFE/upscale/test can run immediately...")
    # safe ffmpeg drawtext value (escape single quotes)
    txt = cfg["prompt"][:80].replace("'", "")
    ffmpeg_cmd = (
        f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x101018 "
        f"-t 2 -vf drawtext=text='{txt}':fontsize=28:fontcolor=white:x=20:y=40 "
        f"'{out_mp4}'"
    )
    res = subprocess.run(ffmpeg_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if res.returncode == 0:
        print("✅ Simulated SVD output created at:", out_mp4)
    else:
        print("⚠️ ffmpeg failed to create simulated output. stderr:")
        print(res.stderr[:2000])
        raise RuntimeError("ffmpeg simulation failed; check ffmpeg availability.")

# ---------- try to run real sample() if available ----------
# Many SVD notebooks expose a `sample()` function in session. We'll attempt to call it safely.
# If sample() isn't available or call fails, we keep the simulated mp4 created above.
sample_called = False
if 'sample' in globals() and callable(globals()['sample']):
    print("Detected sample() in session — attempting to call sample() with job config.")
    try:
        # attempt a generic call signature. This may need adaptation depending on the notebook's sample() signature.
        # We'll read the cfg and try common kwargs. Wrap in try/except to avoid killing the cell on mismatch.
        cfg = json.load(open(job_cfg, 'r', encoding='utf8'))
        # Common safe arguments — you might need to adapt these for your sample()
        kwargs = dict(
            input_path=cfg.get("conditioning_image", ""),
            resize_image=True,
            num_frames=cfg.get("num_frames", 14),
            num_steps=30,
            seed=cfg.get("seed", "random"),
            decoding_t=2,
            fps_id=6,
            motion_bucket_id=127,
            cond_aug=0.02,
            device='cuda' if (torch.cuda.is_available() if 'torch' in globals() else False) else 'cpu',
            skip_filter=True
        )
        print("Calling sample(...) with kwargs (trimmed):", {k: kwargs[k] for k in ['num_frames','seed','device']})
        out_paths = globals()['sample'](**kwargs)
        print("sample() returned:", out_paths)
        sample_called = True
    except Exception as e:
        print("sample() call failed (ok) — will use simulated output. Error:", repr(e))

else:
    print("No sample() found in session — simulated output will be used for now.")

# ---------- final reporting ----------
print("\n--- SUMMARY ---")
print("job_cfg used:", job_cfg)
cfg_obj = json.load(open(job_cfg, 'r', encoding='utf8'))
print("cfg['out'] ->", cfg_obj.get("out"))
if sample_called:
    print("Real sample() executed. Check returned outputs above.")
else:
    print("No real sample() executed. Simulated mp4 exists at cfg['out'] and downstream steps can proceed.")
print("If you want me to replace the simulation with an exact sample() call for your generative-models fork, say: 'Plug real sample()' and paste the notebook sampling function cell or give me the exact repo/branch you used.")


Looking for existing SVD job cfg in /tmp ...
Found job config: /tmp/svd_job_1763661681_0e6ba7/cfg.json
No sample() found in session — simulated output will be used for now.

--- SUMMARY ---
job_cfg used: /tmp/svd_job_1763661681_0e6ba7/cfg.json
cfg['out'] -> /tmp/svd_job_1763661681_0e6ba7/svd_job_1763661681_0e6ba7.mp4
No real sample() executed. Simulated mp4 exists at cfg['out'] and downstream steps can proceed.
If you want me to replace the simulation with an exact sample() call for your generative-models fork, say: 'Plug real sample()' and paste the notebook sampling function cell or give me the exact repo/branch you used.


In [29]:
# Advanced, robust MP4 repair + interpolation cell (Colab-ready)
# Paste and run in a fresh Colab Python cell.

import os, subprocess, sys, shutil, time, uuid
from pathlib import Path

# ---------- Config (edit only if you know what you're doing) ----------
# fallback user-uploaded file (from your session)
UPLOADED_SAMPLE = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"

# output folder for processed files
OUT_DIR = Path("/tmp/processed_svd_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------- helpers ----------
def run(cmd, capture=True):
    """Run shell command; return (rc, stdout, stderr)."""
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return p.returncode, p.stdout or "", p.stderr or ""

def safe_print_head(tag, txt, n=800):
    print(f"\n[{tag}]")
    if not txt:
        print(" <empty>")
    else:
        print(txt[:n] + ("...\n" if len(txt) > n else "\n"))

# ensure fonts for drawtext exist (silently ignore errors)
print("Ensuring fonts available for ffmpeg drawtext...")
rc, out, err = run("fc-list : file | grep -i dejavu || true")
if not out:
    _ = run("apt-get update -qq && apt-get install -y -qq fonts-dejavu-core || true")
    rc, out, err = run("fc-list : file | grep -i dejavu || true")
print("Fonts check done.")

# ---------- find candidate input mp4 ----------
print("\nSearching for candidate SVD outputs in /tmp ...")
candidates = sorted(Path("/tmp").glob("svd_*/*.mp4"), key=lambda p: p.stat().st_mtime, reverse=True)
input_path = None
if candidates:
    input_path = candidates[0]
    print("Found candidate:", str(input_path))
else:
    # fallback to the known uploaded sample file
    if Path(UPLOADED_SAMPLE).exists():
        print("No /tmp svd outputs found. Falling back to uploaded sample:", UPLOADED_SAMPLE)
        # copy it into a job-like location so downstream code expects /tmp/svd_*/file.mp4
        jobdir = Path("/tmp/svd_fallback_" + uuid.uuid4().hex[:6])
        jobdir.mkdir(parents=True, exist_ok=True)
        input_path = jobdir / "fallback_input.mp4"
        shutil.copy2(UPLOADED_SAMPLE, input_path)
        print("Copied uploaded sample to:", str(input_path))
    else:
        raise FileNotFoundError("No SVD outputs found in /tmp and uploaded sample missing at: " + UPLOADED_SAMPLE)

# convert to Path object
input_path = Path(input_path)
print("Using input:", input_path)

# ---------- validate file size ----------
min_size_bytes = 1024  # tiny threshold
if not input_path.exists() or input_path.stat().st_size < min_size_bytes:
    print("Input missing or too small; creating a valid placeholder video at the input path.")
    # create placeholder valid mp4 with faststart
    placeholder_cmd = (
        f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x101018 -t 2 "
        f"-vf drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf:text='SIMULATED SVD':fontsize=28:fontcolor=white:x=20:y=40 "
        f"-c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{input_path}'"
    )
    rc, out, err = run(placeholder_cmd)
    safe_print_head("ffmpeg-create-placeholder-stdout", out)
    safe_print_head("ffmpeg-create-placeholder-stderr", err)
    if rc != 0:
        raise RuntimeError("Failed to create placeholder mp4. See stderr above.")

# ---------- probe input with ffprobe ----------
print("\nProbing input with ffprobe...")
rc, out, err = run(f"ffprobe -v error -show_format -show_streams '{input_path}'")
safe_print_head("ffprobe", out + err, 1200)
# detect 'moov' related errors heuristically
if ("moov" in err.lower()) or ("invalid data found" in err.lower()) or rc != 0:
    print("Detected container issues (moov/invalid). Attempting to remux/recreate clean mp4 with -movflags +faststart.")
    fixed = input_path.with_suffix(".fixed.mp4")
    remux_cmd = f"ffmpeg -y -i '{input_path}' -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{fixed}'"
    rc, out, err = run(remux_cmd)
    safe_print_head("ffmpeg-remux-stdout", out)
    safe_print_head("ffmpeg-remux-stderr", err)
    if rc == 0 and fixed.exists():
        # replace original safely (keep backup)
        backup = input_path.with_suffix(".bak.mp4")
        shutil.move(str(input_path), str(backup))
        shutil.move(str(fixed), str(input_path))
        print("Remux successful. Replaced input with fixed file. Backup at", str(backup))
    else:
        print("Remux failed. We'll attempt to recreate a clean placeholder input instead.")
        recreate_cmd = (
            f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x101018 -t 2 "
            f"-vf drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf:text='SIM RECREATE':fontsize=28:fontcolor=white:x=20:y=40 "
            f"-c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{input_path}'"
        )
        rc, out, err = run(recreate_cmd)
        safe_print_head("ffmpeg-recreate-stderr", err)
        if rc != 0:
            raise RuntimeError("Failed to remux or recreate a valid input mp4. Stderr above.")

# ---------- check ffmpeg supports minterpolate ----------
print("\nChecking ffmpeg filters for 'minterpolate' support...")
rc, out, err = run("ffmpeg -hide_banner -filters")
supports_minterp = "minterpolate" in out
print("minterpolate supported?" , supports_minterp)

# ---------- attempt interpolation (minterpolate) with fallback to fps re-encode ----------
base_name = input_path.stem
rife_out = OUT_DIR / f"{base_name}_rife.mp4"
fps_out = OUT_DIR / f"{base_name}_fps30.mp4"

if supports_minterp:
    print("Running minterpolate interpolation (this may be slow)...")
    # use quotes carefully
    cmd_minterp = (
        f"ffmpeg -y -i '{input_path}' -filter:v \"minterpolate=fps=30:mi_mode=mci:mc_mode=aobmc:me_mode=bidir:vsbmc=1\" "
        f"-c:v libx264 -pix_fmt yuv420p -crf 18 '{rife_out}'"
    )
    rc, out, err = run(cmd_minterp, )
    safe_print_head("minterpolate-stdout", out)
    safe_print_head("minterpolate-stderr", err, 4000)
    if rc == 0 and rife_out.exists():
        final_out = rife_out
        print("minterpolate succeeded ->", str(final_out))
    else:
        print("minterpolate failed — falling back to fps re-encode.")
        rc, out, err = run(f"ffmpeg -y -i '{input_path}' -vf fps=30 -c:v libx264 -pix_fmt yuv420p -crf 18 '{fps_out}'")
        safe_print_head("fps-stdout", out)
        safe_print_head("fps-stderr", err)
        if rc == 0 and fps_out.exists():
            final_out = fps_out
        else:
            raise RuntimeError("Both minterpolate and fps fallback failed. See stderr above.")
else:
    print("minterpolate not supported in this ffmpeg build — using fps re-encode fallback (safe).")
    rc, out, err = run(f"ffmpeg -y -i '{input_path}' -vf fps=30 -c:v libx264 -pix_fmt yuv420p -crf 18 '{fps_out}'")
    safe_print_head("fps-stdout", out)
    safe_print_head("fps-stderr", err)
    if rc == 0 and fps_out.exists():
        final_out = fps_out
    else:
        raise RuntimeError("fps re-encode fallback failed. See stderr above.")

# ---------- final report ----------
print("\n--- DONE ---")
print("Input used:", str(input_path))
print("Final interpolation output:", str(final_out))
print("Filesize (bytes):", final_out.stat().st_size if final_out.exists() else "MISSING")
print("You can now use this file for RIFE/upscale/stitch steps.")

# show quick ls of output dir
print("\nOUT_DIR listing:")
for p in sorted(OUT_DIR.iterdir(), key=lambda x: x.stat().st_mtime):
    print(p.name, "-", p.stat().st_size, "bytes")


Ensuring fonts available for ffmpeg drawtext...
Fonts check done.

Searching for candidate SVD outputs in /tmp ...
Found candidate: /tmp/svd_job_1763661681_0e6ba7/svd_job_1763661681_0e6ba7.mp4
Using input: /tmp/svd_job_1763661681_0e6ba7/svd_job_1763661681_0e6ba7.mp4

Probing input with ffprobe...

[ffprobe]
[STREAM]
index=0
codec_name=h264
codec_long_name=H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10
profile=High
codec_type=video
codec_tag_string=avc1
codec_tag=0x31637661
width=720
height=1280
coded_width=720
coded_height=1280
closed_captions=0
has_b_frames=2
sample_aspect_ratio=1:1
display_aspect_ratio=9:16
pix_fmt=yuv420p
level=31
color_range=unknown
color_space=unknown
color_transfer=unknown
color_primaries=unknown
chroma_location=left
field_order=unknown
refs=1
is_avc=true
nal_length_size=4
id=N/A
r_frame_rate=6/1
avg_frame_rate=6/1
time_base=1/12288
start_pts=0
start_time=0.000000
duration_ts=24576
duration=2.000000
bit_rate=12144
max_bit_rate=N/A
bits_per_raw_sample=8
nb_frames=12
nb

In [28]:
import gradio as gr

def one_click(prompt_text):
    # TEXT-ONLY ! generate SVD config
    out, jobdir = generate_svd_from_text(prompt_text, seed="random", num_frames=25, out_prefix="oneclick")
    # RUN sampling cell manually (or we can call sample() if present)
    # For now we simulate generation (same logic as SVD sampling cell)
    run_cmd(f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x331122 -t 3 -vf drawtext=\"text='ONECLICK {prompt_text[:30]}':fontsize=32:fontcolor=white:x=20:y=20\" {out}")
    # postprocess
    rife = interpolate_with_rife(out)
    up = upscale_realesrgan(rife, scale=1)
    final = "/content/oneclick_final.mp4"
    stitch_files([up], final)
    voice = tts_generate(prompt_text, outfile="/content/oneclick_voice.wav")
    run_cmd(f"ffmpeg -y -i '{final}' -i '{voice}' -c:v copy -c:a aac -shortest '/content/oneclick_final_audio.mp4'")
    dst = os.path.join(OUTPUT_DRIVE_FOLDER, "oneclick_final.mp4")
    shutil.copy("/content/oneclick_final_audio.mp4", dst)
    return dst

demo = gr.Interface(fn=one_click, inputs=gr.Textbox(lines=2, placeholder="Enter prompt..."), outputs=gr.File())
demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [27]:
# REAL SVD SAMPLING runner (run only if you have checkpoints & optional HF token)
import os, json, subprocess
from pathlib import Path

# EDIT these if your checkpoints live in Drive
CHECKPOINT_DIR = "/content/checkpoints"   # <-- change if your checkpoints are in Drive
SVD_VERSION = "svd"  # "svd" or "svd-xt-1-1"

# find latest job cfg
jobcfg = None
for d in sorted(Path("/tmp").glob("svd_*"), key=lambda x: x.stat().st_mtime, reverse=True):
    c = d / "cfg.json"
    if c.exists():
        jobcfg = c
        break
if not jobcfg:
    raise RuntimeError("No job cfg found in /tmp. Run TEXT-ONLY / IMAGE-TO-VIDEO cell first.")

cfg = json.load(open(jobcfg,"r"))
print("Using job cfg:", jobcfg)
print("cfg:", {k:cfg[k] for k in ("prompt","seed","num_frames","out") if k in cfg})

# sample() attempt
if 'sample' in globals() and callable(globals()['sample']):
    print("sample() found in session — calling sample() now...")
    try:
        out = sample(
            input_path=cfg.get("conditioning_image",""),
            resize_image=True,
            num_frames=cfg.get("num_frames",14),
            num_steps=30,
            seed=cfg.get("seed","random"),
            decoding_t=2,
            fps_id=6,
            motion_bucket_id=127,
            cond_aug=0.02,
            device='cuda' if (("torch" in globals()) and torch.cuda.is_available()) else 'cpu',
            skip_filter=True
        )
        print("sample() returned:", out)
    except Exception as e:
        print("sample() call failed. Error:", e)
        print("If it failed due to missing imports / checkpoints, ensure CHECKPOINT_DIR contains SVD weights and you loaded model cells.")
else:
    print("No sample() defined in this session. If you want real SVD, either run a notebook cell that loads SVD 'sample' function or paste the sampling cell. Otherwise use the post-process cell (option 2).")


Using job cfg: /tmp/svd_job_1763667496_42b842/cfg.json
cfg: {'prompt': 'cat', 'seed': 1234, 'num_frames': 25, 'out': '/tmp/svd_job_1763667496_42b842/svd_job_1763667496_42b842.mp4'}
No sample() defined in this session. If you want real SVD, either run a notebook cell that loads SVD 'sample' function or paste the sampling cell. Otherwise use the post-process cell (option 2).


In [50]:
# One-run, robust post-process cell — no parentheses in shell filters (Colab-ready)
import subprocess, shutil, json, time
from pathlib import Path

# Config (use your uploaded file)
UPLOADED = Path("/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4")
OUT_DIR = Path("/tmp/processed_svd_outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)
DRIVE_OUT = Path("/content/drive/MyDrive/AI-Automation/outputs"); DRIVE_OUT.mkdir(parents=True, exist_ok=True)

def run(cmd, fail_ok=False):
    print("RUN:", cmd)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.stdout: print(p.stdout[:1500])
    if p.stderr: print("ERR:", p.stderr[:1500])
    if p.returncode != 0 and not fail_ok:
        raise RuntimeError(f"Command failed: {cmd}\nstderr snippet above")
    return p

# 1) Find latest SVD output; fallback to uploaded
cands = sorted(Path("/tmp").glob("svd_*/*.mp4"), key=lambda p: p.stat().st_mtime, reverse=True)
if cands:
    input_mp4 = cands[0]
else:
    if UPLOADED.exists():
        jobdir = Path("/tmp/svd_fallback_"+str(int(time.time())))
        jobdir.mkdir(parents=True, exist_ok=True)
        input_mp4 = jobdir / "fallback_input.mp4"
        shutil.copy2(str(UPLOADED), str(input_mp4))
    else:
        raise FileNotFoundError("No SVD outputs in /tmp and uploaded sample missing.")

print("Using input:", input_mp4)

# 2) Ensure valid container (remux with movflags +faststart) -> remuxed
remuxed = OUT_DIR / f"{input_mp4.stem}_remux.mp4"
run(f"ffmpeg -y -i '{input_mp4}' -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{remuxed}'", fail_ok=True)
if not remuxed.exists():
    # try recreate using first frame (guaranteed compatible)
    frame = OUT_DIR / f"{input_mp4.stem}_frame.png"
    run(f"ffmpeg -y -i '{input_mp4}' -vframes 1 '{frame}'", fail_ok=True)
    run(f"ffmpeg -y -loop 1 -i '{frame}' -t 2 -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{remuxed}'", fail_ok=False)

# 3) Interpolate (minterpolate) with fallback to fps re-encode
filters = subprocess.run("ffmpeg -hide_banner -filters", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True).stdout
supports_minterp = "minterpolate" in (filters or "")
interp_out = OUT_DIR / f"{input_mp4.stem}_interp.mp4"
if supports_minterp:
    try:
        run(f"ffmpeg -y -i '{remuxed}' -filter:v \"minterpolate=fps=30:mi_mode=mci:mc_mode=aobmc:me_mode=bidir:vsbmc=1\" -c:v libx264 -pix_fmt yuv420p -crf 18 '{interp_out}'", fail_ok=False)
    except Exception as e:
        print("minterpolate failed, falling back to fps re-encode. Error:", e)
        run(f"ffmpeg -y -i '{remuxed}' -vf fps=30 -c:v libx264 -pix_fmt yuv420p -crf 18 '{interp_out}'")
else:
    run(f"ffmpeg -y -i '{remuxed}' -vf fps=30 -c:v libx264 -pix_fmt yuv420p -crf 18 '{interp_out}'")

if not interp_out.exists():
    interp_out = remuxed

# 4) Read width/height of interp_out using ffprobe (to compute numeric pad offsets)
def probe_wh(path):
    p = subprocess.run(f"ffprobe -v error -select_streams v:0 -show_entries stream=width,height -of json '{path}'", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    try:
        info = json.loads(p.stdout)
        st = info.get("streams", [{}])[0]
        return int(st.get("width", 0)), int(st.get("height", 0))
    except Exception:
        return 0, 0

iw, ih = probe_wh(str(interp_out))
print("Detected size:", iw, "x", ih)
target_w, target_h = 1080, 1920

# If input is larger than target, we will scale-down preserving aspect ratio and then pad; compute scaled dimensions by ffmpeg scale math in Python
# calculate scale factor to fit into target
if iw == 0 or ih == 0:
    # fallback: just re-encode to target size directly (safe)
    up_out = OUT_DIR / f"{interp_out.stem}_up.mp4"
    run(f"ffmpeg -y -i '{interp_out}' -vf scale={target_w}:{target_h} -c:v libx264 -pix_fmt yuv420p -crf 18 '{up_out}'")
else:
    # scale to fit target while preserving aspect
    # compute scale factor
    scale_w = target_w / iw
    scale_h = target_h / ih
    scale_factor = min(scale_w, scale_h)
    new_w = int(iw * scale_factor)
    new_h = int(ih * scale_factor)
    pad_x = max((target_w - new_w) // 2, 0)
    pad_y = max((target_h - new_h) // 2, 0)
    up_out = OUT_DIR / f"{interp_out.stem}_up.mp4"
    vf = f"scale={new_w}:{new_h},pad={target_w}:{target_h}:{pad_x}:{pad_y}"
    # safe: no parentheses in vf string
    run(f"ffmpeg -y -i '{interp_out}' -vf \"{vf}\" -c:v libx264 -pix_fmt yuv420p -crf 18 '{up_out}'")

if not up_out.exists():
    up_out = interp_out

print("Upscaled output:", up_out)

# 5) Make ~15s by concat repetitions (use concat demuxer)
def duration(path):
    p = subprocess.run(f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 '{path}'", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    try:
        return float(p.stdout.strip())
    except:
        return 0.0

cur_dur = duration(str(up_out))
target = 15.0
if cur_dur < target and cur_dur > 0:
    reps = int((target // cur_dur) + 1)
    listfile = OUT_DIR / "concat_list.txt"
    with open(listfile, "w") as f:
        for i in range(reps):
            f.write(f"file '{up_out}'\n")
    stitched = OUT_DIR / f"{up_out.stem}_stitched.mp4"
    run(f"ffmpeg -y -f concat -safe 0 -i '{listfile}' -c copy '{stitched}'")
    final_video = stitched if stitched.exists() else up_out
else:
    final_video = up_out

print("Final video (pre-audio):", final_video)

# 6) TTS (gTTS) and merge (safe)
try:
    run("python -m pip install -q gTTS pydub", fail_ok=True)
    from gtts import gTTS
    tts_text = "Tiny mechanical fox explores a sunlit garden."
    tts_file = OUT_DIR / "voice.mp3"
    gTTS(tts_text, lang='en').save(str(tts_file))
    merged = OUT_DIR / f"{final_video.stem}_final_audio.mp4"
    run(f"ffmpeg -y -i '{final_video}' -i '{tts_file}' -c:v copy -c:a aac -shortest '{merged}'")
    final_with_audio = merged if merged.exists() else final_video
except Exception as e:
    print("TTS failed, continuing without audio. Error:", e)
    final_with_audio = final_video

# 7) burn captions — use numeric x,y to avoid parentheses
caption_text = "Tiny mechanical fox explores a sunlit garden."
captioned = OUT_DIR / f"{final_with_audio.stem}_captioned.mp4"
# choose numeric x (left indent) and numeric y from height
y_pos = max(target_h - 150, 10)
run(f"ffmpeg -y -i '{final_with_audio}' -vf \"drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf:text='{caption_text}':fontsize=40:fontcolor=white:box=1:boxcolor=0x00000099:x=40:y={y_pos}\" -c:a copy '{captioned}'", fail_ok=True)
if not captioned.exists():
    captioned = final_with_audio

# 8) copy result to Drive
timestamp = int(time.time())
dst = DRIVE_OUT / f"final_short_{timestamp}.mp4"
shutil.copy2(str(captioned), str(dst))

print("\n--- DONE ---")
print("Final short saved to Drive:", dst)
print("Local file:", captioned)


Using input: /tmp/svd_job_1763661681_0e6ba7/svd_job_1763661681_0e6ba7.mp4
RUN: ffmpeg -y -i '/tmp/svd_job_1763661681_0e6ba7/svd_job_1763661681_0e6ba7.mp4' -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '/tmp/processed_svd_outputs/svd_job_1763661681_0e6ba7_remux.mp4'
ERR: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --

In [55]:
from pathlib import Path
import shutil

# 1) find your uploaded MP4 in /content
mp4s = list(Path("/content").glob("*.mp4"))
print("Found MP4s:", mp4s)

if not mp4s:
    raise FileNotFoundError("❌ No MP4 found in /content. Upload again bro.")

UPLOADED_SAMPLE = mp4s[0]
print("Using UPLOADED_SAMPLE =", UPLOADED_SAMPLE)


Found MP4s: [PosixPath('/content/sample_from_upload.mp4')]
Using UPLOADED_SAMPLE = /content/sample_from_upload.mp4


In [56]:
BACKUP = Path("/content/drive/MyDrive/AI-Automation/last_uploaded_sample.mp4")
shutil.copy2(UPLOADED_SAMPLE, BACKUP)
print("Backed up to:", BACKUP)


Backed up to: /content/drive/MyDrive/AI-Automation/last_uploaded_sample.mp4


In [57]:
from pathlib import Path
import shutil

BACKUP = Path("/content/drive/MyDrive/AI-Automation/last_uploaded_sample.mp4")
RESTORE = Path("/content/last_uploaded_sample.mp4")

if BACKUP.exists():
    shutil.copy2(BACKUP, RESTORE)
    UPLOADED_SAMPLE = RESTORE
else:
    UPLOADED_SAMPLE = None

print("Restored sample:", UPLOADED_SAMPLE)


Restored sample: /content/last_uploaded_sample.mp4


In [60]:
# AUTO-FIX uploaded-sample path (one cell) — run now
from pathlib import Path
import shutil, os, time

# Candidate paths (tries these in order)
candidates = [
    Path("/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"),
    Path("/content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"),
    Path("/content/drive/MyDrive/AI-Automation/sample.mp4"),
    Path("/content/last_uploaded_sample.mp4"),
]

found = None
for p in candidates:
    try:
        if p.exists():
            found = p
            break
    except Exception:
        continue

# If not found, try scanning common folders for any mp4
if not found:
    for root in [Path("/content"), Path("/mnt"), Path("/content/drive/MyDrive/AI-Automation")]:
        if root.exists():
            mp4s = list(root.rglob("*.mp4"))
            if mp4s:
                found = mp4s[0]
                break

if not found:
    raise FileNotFoundError("No MP4 found in known locations (/mnt/data, /content, Drive). Upload a sample or put it in Drive:/MyDrive/AI-Automation/sample.mp4")

# Ensure /content exists and copy into it (so pipeline uses /content path)
content_target = Path("/content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4")
try:
    # Only copy if source != target or target missing
    if found.resolve() != content_target.resolve():
        shutil.copy2(str(found), str(content_target))
except Exception as e:
    print("Warning copying to /content:", e)

# Back up into Drive persistent location
drive_path = Path("/content/drive/MyDrive/AI-Automation/last_uploaded_sample.mp4")
try:
    # create Drive folder if missing
    drive_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(str(content_target), str(drive_path))
    print("Backed up sample to Drive:", drive_path)
except Exception as e:
    print("Warning backing up to Drive (maybe Drive not mounted):", e)

# Final variable for rest of notebook
UPLOADED_SAMPLE = content_target
print("Final UPLOADED_SAMPLE set to:", UPLOADED_SAMPLE)
print("Exists:", UPLOADED_SAMPLE.exists(), " Size (MB):", (UPLOADED_SAMPLE.stat().st_size/1024/1024) if UPLOADED_SAMPLE.exists() else "N/A")

# quick sanity: if file missing after copy, show found path for debugging
if not UPLOADED_SAMPLE.exists():
    print("Debug - originally found at:", found)
    raise FileNotFoundError(f"Failed to ensure uploaded sample at {UPLOADED_SAMPLE}. Found candidate: {found}")

# optional small wait to ensure FS flush
time.sleep(0.5)


Backed up sample to Drive: /content/drive/MyDrive/AI-Automation/last_uploaded_sample.mp4
Final UPLOADED_SAMPLE set to: /content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4
Exists: True  Size (MB): 0.05820941925048828


In [64]:
# KlingAI compact Gradio UI — 3 previews, Load Model, Generate 3, compact styling
# Paste & run in Colab after mounting Drive (drive.mount('/content/drive', force_remount=True))
import os, subprocess, shutil, time, uuid, json
from pathlib import Path
import gradio as gr

# ---------- CONFIG ----------
DRIVE_OUT = Path("/content/drive/MyDrive/AI-Automation/outputs")
DRIVE_OUT.mkdir(parents=True, exist_ok=True)

# IMPORTANT: your uploaded sample (session path). Use this path in tools.
FALLBACK_SAMPLE_PATH = "/content/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"
FALLBACK_SAMPLE = Path(FALLBACK_SAMPLE_PATH)

TMP_BASE = Path("/tmp/kling_ui"); TMP_BASE.mkdir(parents=True, exist_ok=True)

# ---------- helper to run shell cmds ----------
def run(cmd, fail_ok=False):
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout = p.stdout.strip()
    stderr = p.stderr.strip()
    if stdout:
        print(stdout[:2000])
    if stderr:
        print("ERR:", stderr[:3000])
    if p.returncode != 0 and not fail_ok:
        raise RuntimeError(f"Command failed: {cmd}\n{stderr[:2000]}")
    return p

# ---------- Model loader (attempts safe non-blocking install) ----------
MODEL_READY = {"ok": False, "msg": "Model not loaded."}
def load_svd_model():
    """
    Try to prepare environment & load stable-video-diffusion code.
    This will:
     - clone generative-models (if missing)
     - pip install common deps (light)
     - check Drive for checkpoints at /content/drive/MyDrive/AI-Automation/checkpoints
     - attempt to import sample() from generative-models (best-effort)
    NOTE: Gated weights need manual placement in Drive.
    """
    try:
        # 1) clone if missing
        if not Path("generative-models").exists():
            run("git clone https://github.com/Stability-AI/generative-models.git generative-models", fail_ok=True)
        # 2) install minimal deps to make imports work (non-exhaustive)
        run("python -m pip install -q -U torch torchvision --index-url https://download.pytorch.org/whl/cu118", fail_ok=True)
        run("python -m pip install -q omegaconf einops accelerate transformers timm open-clip-torch imwatermark", fail_ok=True)
        # 3) try to import sample() from the repo (in-process)
        import sys
        if "generative-models" not in sys.path:
            sys.path.append("generative-models")
        # best-effort import (may still fail if many deps missing)
        try:
            from sgm.sampling import sample as svd_sample  # path depends on fork; try canonical
            globals()['sample'] = svd_sample
            MODEL_READY['ok'] = True
            MODEL_READY['msg'] = "Loaded sample() from generative-models (best-effort)."
            return ("SVD model function loaded (best-effort). If you plan to run large gen, ensure weights are in Drive:/AI-Automation/checkpoints and restart runtime.", True)
        except Exception as e:
            # If import fail, still return instructive message
            MODEL_READY['ok'] = False
            MODEL_READY['msg'] = f"Could not import sample(): {e}"
            return (f"Partial setup done. sample() not importable: {e}\nPlace SVD checkpoints in Drive:/AI-Automation/checkpoints and follow SVD notebook instructions, or ask me to fully script weight download.", False)
    except Exception as e:
        return (f"Load failed: {e}", False)

# ---------- Post-process (compact, same robust pipeline with duration) ----------
def post_process(input_mp4_path, prompt_text, duration_sec:int):
    OUT_DIR = TMP_BASE / "processed"; shutil.rmtree(OUT_DIR, ignore_errors=True); OUT_DIR.mkdir(parents=True, exist_ok=True)
    input_mp4 = Path(input_mp4_path)
    # remux / ensure valid container
    remuxed = OUT_DIR / (input_mp4.stem + "_remux.mp4")
    run(f"ffmpeg -y -i '{input_mp4}' -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{remuxed}'", fail_ok=True)
    if not remuxed.exists():
        # fallback from frame
        frame = OUT_DIR / "fallback_frame.png"
        run(f"ffmpeg -y -i '{input_mp4}' -vframes 1 '{frame}'", fail_ok=True)
        run(f"ffmpeg -y -loop 1 -i '{frame}' -t 2 -c:v libx264 -pix_fmt yuv420p -crf 18 -preset veryfast -movflags +faststart '{remuxed}'", fail_ok=False)

    # simple upsampling/fps -> use fps 30
    interp_out = OUT_DIR / (input_mp4.stem + "_interp.mp4")
    run(f"ffmpeg -y -i '{remuxed}' -vf fps=30 -c:v libx264 -pix_fmt yuv420p -crf 18 '{interp_out}'", fail_ok=True)
    if not interp_out.exists():
        interp_out = remuxed

    # scale/pad to 1080x1920 numerically
    def probe_wh(path):
        p = subprocess.run(f"ffprobe -v error -select_streams v:0 -show_entries stream=width,height -of json '{path}'", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        try:
            info = json.loads(p.stdout); st = info.get("streams", [{}])[0]; return int(st.get("width",0)), int(st.get("height",0))
        except:
            return 0,0
    iw, ih = probe_wh(str(interp_out)); target_w, target_h = 1080,1920
    up_out = OUT_DIR / (interp_out.stem + "_up.mp4")
    if iw==0 or ih==0:
        run(f"ffmpeg -y -i '{interp_out}' -vf scale={target_w}:{target_h} -c:v libx264 -pix_fmt yuv420p -crf 18 '{up_out}'", fail_ok=True)
    else:
        scale_w = target_w/iw; scale_h=target_h/ih; sf=min(scale_w,scale_h)
        new_w = max(1,int(iw*sf)); new_h=max(1,int(ih*sf))
        pad_x = max((target_w-new_w)//2,0); pad_y = max((target_h-new_h)//2,0)
        vf = f"scale={new_w}:{new_h},pad={target_w}:{target_h}:{pad_x}:{pad_y}"
        run(f"ffmpeg -y -i '{interp_out}' -vf \"{vf}\" -c:v libx264 -pix_fmt yuv420p -crf 18 '{up_out}'", fail_ok=True)
    if not up_out.exists(): up_out = interp_out

    # ensure exact duration: repeat/trim
    def get_dur(path):
        p = subprocess.run(f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 '{path}'", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        try: return float(p.stdout.strip())
        except: return 0.0
    cur = get_dur(str(up_out)); final_video = up_out
    if cur>0 and cur < duration_sec:
        reps = int((duration_sec//cur)+1)
        lf = OUT_DIR / "concat.txt"
        with open(lf,"w") as f:
            for _ in range(reps): f.write(f"file '{up_out}'\n")
        stitched = OUT_DIR / (up_out.stem + "_stitched.mp4")
        run(f"ffmpeg -y -f concat -safe 0 -i '{lf}' -c copy '{stitched}'", fail_ok=True)
        final_video = stitched if stitched.exists() else up_out
    elif cur>duration_sec:
        trimmed = OUT_DIR / (up_out.stem + "_trim.mp4")
        run(f"ffmpeg -y -i '{up_out}' -t {duration_sec} -c:v libx264 -pix_fmt yuv420p -crf 18 '{trimmed}'", fail_ok=True)
        final_video = trimmed if trimmed.exists() else up_out

    # tts + merge audio
    try:
        run("python -m pip install -q gTTS pydub", fail_ok=True)
        from gtts import gTTS
        tts_file = OUT_DIR / "voice.mp3"
        tts_text = prompt_text or "Tiny AI short."
        gTTS(tts_text, lang='en').save(str(tts_file))
        merged = OUT_DIR / (final_video.stem + "_final_audio.mp4")
        run(f"ffmpeg -y -i '{final_video}' -i '{tts_file}' -c:v copy -c:a aac -shortest '{merged}'", fail_ok=True)
        final_with_audio = merged if merged.exists() else final_video
    except Exception:
        final_with_audio = final_video

    # captions
    captioned = OUT_DIR / (final_with_audio.stem + "_captioned.mp4")
    y_pos = max(target_h - 150, 10)
    run(f"ffmpeg -y -i '{final_with_audio}' -vf \"drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf:text='{prompt_text}':fontsize=36:fontcolor=white:box=1:boxcolor=0x00000099:x=30:y={y_pos}\" -c:a copy '{captioned}'", fail_ok=True)
    final = captioned if captioned.exists() else final_with_audio
    ts = int(time.time()); dst = DRIVE_OUT / f"final_short_{ts}.mp4"; shutil.copy2(str(final), str(dst))
    return str(dst), str(final)

# ---------- Generate action: produce three videos and return previews & paths ----------
def generate_three(mode, prompt, image, duration_choice, custom_duration):
    logs = []
    try:
        # resolve duration
        if duration_choice == "Custom":
            try: duration_sec = max(1, min(60, int(custom_duration)))
            except: return ("Invalid custom duration", "", "", "", "", "", "")
        else:
            duration_sec = int(duration_choice)

        # try SVD sample() if loaded
        svd_paths = []
        if 'sample' in globals() and callable(globals()['sample']):
            for i in range(3):
                logs.append(f"Running SVD sample() attempt {i+1}...")
                try:
                    out = globals()['sample'](
                        input_path=str(image) if (mode=="IMAGE" and image) else "",
                        resize_image=True,
                        num_frames=25,
                        num_steps=30,
                        seed="random",
                        decoding_t=2,
                        fps_id=6,
                        motion_bucket_id=127,
                        cond_aug=0.02,
                        device='cuda' if ('torch' in globals() and __import__('torch').cuda.is_available()) else 'cpu',
                        skip_filter=True
                    )
                    if isinstance(out, (list,tuple)) and out:
                        svd_paths.append(str(out[0]))
                    elif isinstance(out,str):
                        svd_paths.append(out)
                except Exception as e:
                    logs.append(f"SVD run failed: {e}")
                    break

        # fallback: copy sample into job files for each of the 3 slots
        if len(svd_paths) < 3:
            base_candidate = Path("/content/drive/MyDrive/AI-Automation/last_uploaded_sample.mp4")
            if base_candidate.exists():
                fallback_src = base_candidate
                logs.append("Using Drive backup sample")
            elif FALLBACK_SAMPLE.exists():
                fallback_src = FALLBACK_SAMPLE
                logs.append("Using session fallback sample")
            else:
                mp4s = list(Path("/content").glob("*.mp4"))
                if mp4s:
                    fallback_src = mp4s[0]; logs.append("Using /content mp4 fallback")
                else:
                    return ("No sample available. Upload sample to /content or Drive:/AI-Automation/last_uploaded_sample.mp4", "", "", "", "", "", "")

            # create three job copies
            for i in range(3):
                job_out = str(TMP_BASE / f"sim_job_{int(time.time())}_{i}.mp4")
                shutil.copy2(str(fallback_src), job_out)
                svd_paths.append(job_out)

        previews = []
        final_paths = []
        for idx, p in enumerate(svd_paths[:3]):
            logs.append(f"Post-processing video {idx+1} ...")
            drive_dst, local_final = post_process(p, prompt, duration_sec)
            previews.append(local_final)      # local path preview
            final_paths.append(drive_dst)     # saved path in Drive

        return ("\n".join(logs), previews[0] if previews else "", previews[1] if len(previews)>1 else "", previews[2] if len(previews)>2 else "", final_paths[0] if final_paths else "", final_paths[1] if len(final_paths)>1 else "", final_paths[2] if len(final_paths)>2 else "")
    except Exception as e:
        return (f"ERROR: {e}", "", "", "", "", "", "")

# ---------- Compact KlingAI styling (CSS) ----------
css = """
.kling-card { border-radius:10px; padding:12px; box-shadow:0 6px 18px rgba(0,0,0,0.12); background:linear-gradient(180deg,#0f1724, #071124); color:#e6eef8 }
.kling-small { font-size:13px }
.kling-btn { background:#0ea5a4; color:white; border-radius:8px; padding:8px 12px }
"""

with gr.Blocks(css=css, title="KlingAI Mini", theme=gr.themes.Soft()) as demo:
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("<h3 style='margin:0;color:#c7f9f2'>KlingAI — TinyShort Lab</h3>")
            gr.Markdown("<div class='kling-small'>Compact UI • Load model • Generate x3 • 3 previews • Drive outputs</div>")
            mode = gr.Radio(["TEXT","IMAGE"], value="TEXT", label="Mode", elem_id="mode")
            prompt = gr.Textbox(lines=2, placeholder="woolen cat playing", label="Prompt")
            img_in = gr.Image(type="pil", label="Conditioning image (IMAGE mode)")
            with gr.Row():
                duration_choice = gr.Dropdown(["5","10","15","20","30","Custom"], value="15", label="Duration (sec)")
                custom_duration = gr.Textbox(value="15", label="Custom sec (1-60)")
            with gr.Row():
                load_btn = gr.Button("Load SVD Model", elem_id="loadbtn")
                gen_btn = gr.Button("Generate 3 Shorts", elem_id="genbtn", variant="primary")
            status = gr.Textbox(label="Status / Logs", interactive=False, value="Ready", lines=6)
        with gr.Column(scale=3):
            # previews grid
            gr.Markdown("<div style='display:flex;gap:8px;align-items:flex-start'>")
            preview1 = gr.Video(label="Preview 1")
            preview2 = gr.Video(label="Preview 2")
            preview3 = gr.Video(label="Preview 3")
            gr.Markdown("</div>")
            with gr.Row():
                out1 = gr.Textbox(label="Drive Path 1", interactive=False)
                out2 = gr.Textbox(label="Drive Path 2", interactive=False)
                out3 = gr.Textbox(label="Drive Path 3", interactive=False)
            gr.Markdown("<div class='kling-small' style='margin-top:8px;color:#9be7e5'>Fallback sample used: {}</div>".format(FALLBACK_SAMPLE.name))

    # Bind buttons
    load_btn.click(lambda: load_svd_model(), outputs=[status])
    gen_btn.click(generate_three, inputs=[mode, prompt, img_in, duration_choice, custom_duration], outputs=[status, preview1, preview2, preview3, out1, out2, out3])

demo.launch(share=False, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

ERR: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx -